In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-category-dtypemore-new-features/training_data.csv
/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
!pip install autogluon.tabular[0]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: botocore
    Fo

In [3]:
from sklearn.utils.class_weight import compute_class_weight

train = pd.read_csv('/kaggle/input/all-category-dtypemore-new-features/training_data.csv')
y = train['loan_status']

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
negative_class_count = np.sum(y == 0)
positive_class_count = np.sum(y == 1)

scale_pos_weight = negative_class_count / positive_class_count
print("scale_pos_weight:", scale_pos_weight)

scale_pos_weight: 4.997594997594998


In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

tabular_df = TabularDataset('/kaggle/input/playground-series-s4e10/train.csv')
label = 'loan_status'
features = ['person_age', 'person_income', 'person_home_ownership',
            'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
            'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
            'cb_person_cred_hist_length']
aml = TabularPredictor(label=label, eval_metric='roc_auc', problem_type='binary')

Loaded data from: /kaggle/input/playground-series-s4e10/train.csv | Columns = 13 / 13 | Rows = 58645 -> 58645
No path specified. Models will be saved in: "AutogluonModels/ag-20241022_141221"


In [5]:
hyperparameters = {
    'GBM': {'class_weight': class_weights},
    'CAT': {'scale_pos_weight': scale_pos_weight},
    'XGB': {'scale_pos_weight': scale_pos_weight},
}

aml.fit(train_data=tabular_df, time_limit=39600, presets='best_quality', hyperparameters=hyperparameters)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       30.28 GB / 31.36 GB (96.6%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	

In [6]:
leaderboard = aml.leaderboard()
leaderboard.to_csv('leaderboard.csv', index=False)

In [7]:
leaderboard

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.958225,roc_auc,2.926131,779.902831,0.010723,0.802600,2,True,4
1,LightGBM_BAG_L1,0.956925,roc_auc,0.975490,30.701847,0.975490,30.701847,1,True,1
2,CatBoost_BAG_L1,0.956319,roc_auc,0.097553,560.184674,0.097553,560.184674,1,True,2
3,XGBoost_BAG_L1,0.948889,roc_auc,1.842366,188.213710,1.842366,188.213710,1,True,3


In [8]:
# aml.save('/kaggle/working/best_model')

In [9]:
# import shutil

# shutil.rmtree('/kaggle/working/AutogluonModels')
# os.remove('/kaggle/working/leaderboard.csv')